In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [30]:
# Loading Data
radar_data = pd.read_csv('homework/radar_parameters.csv')

Part 1 - Generating the testing and training data

In [31]:
X = radar_data.drop('R (mm/hr)', axis = 1)
Y = radar_data['R (mm/hr)']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

print(len(x_train), len(x_test))

15175 3794


In [32]:
def base_rain_calc(Zh: np.array) -> np.array:
    z = 10 ** (Zh/10)
    r_baseline = (z/200) ** (1/1.6)

    return r_baseline

In [44]:
baseline_rainrate_test = base_rain_calc(radar_data['Zh (dBZ)'][y_test.index])

Part 2 - Generating a Mulitple Linear Regression Model

In [47]:
model = LinearRegression(fit_intercept=True)

model.fit(x_train,y_train)

lin_reg_R2 = model.score(x_test, y_test)
lin_reg_predicted = model.predict(x_test)
lin_reg_rmse = root_mean_squared_error(y_test, lin_reg_predicted)
baseline_R2 = r2_score(y_test, baseline_rainrate_test)
baseline_rmse = root_mean_squared_error(y_test, baseline_rainrate_test)




In [61]:
print(f"R-squared value for baseline prediction is {np.round(baseline_R2, 2)}")
print(f"RMSE value for baseline prediction is {np.round(baseline_rmse, 2)}")
print(f"R-squared value for multiple linear regression is {np.round(lin_reg_R2, 2)}")
print(f"RMSE value for baseline prediction is {np.round(lin_reg_rmse, 2)}")

R-squared value for baseline prediction is 0.29
RMSE value for baseline prediction is 6.98
R-squared value for multiple linear regression is 0.99
RMSE value for baseline prediction is 0.87
